# Imports

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
import numpy.matlib
import pandas as pd
import time
import os
from os import listdir
from os.path import join, isfile

!pip install jupyter-dash
from jupyter_dash import JupyterDash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import requests
import io
from plotly.subplots import make_subplots

# Data Processing

In [ ]:
path = r'C:\Users\bobby\Documents\Github\PD-Watch-Dashboard\Data\data\110001'

In [ ]:
url = 'https://github.com/BobbyWilt/PD-Watch-Dashboard/raw/main/Data/data/110001/data_segment_1.csv'
test = pd.read_csv(url)

In [ ]:
# test.drop(columns=['Unnamed: 0'], inplace=True)
cols = ['left_acc_x', 'left_acc_y', 'left_acc_z',
                'left_gyr_x', 'left_gyr_y', 'left_gyr_z',
                'right_acc_x', 'right_acc_y', 'right_acc_z',
                'right_gyr_x', 'right_gyr_y', 'right_gyr_z',
                'chest_acc_x', 'chest_acc_y', 'chest_acc_z',
                'chest_gyr_x', 'chest_gyr_y', 'chest_gyr_z'
               ]

In [ ]:
# Loop through files and load csv's into dataframe.  Reset columns and index
for i in range(1, 8):
    locals()[f'df{i}'] = pd.read_csv(os.path.join(path,f'data_segment_{i}.csv'))
    locals()[f'df{i}'].set_index('time', inplace=True)
    locals()[f'df{i}'].columns = cols

# Figures

In [ ]:
# Simple line plot
fig = px.line(df1[['left_acc_x', 'left_acc_y', 'left_acc_z']])
fig.show()

In [ ]:
# multi-line plot - separated by axis
fig = make_subplots(rows=6, cols=1, shared_xaxes=True,
                    vertical_spacing=0.06,
                   subplot_titles=['Accelerometry X-axis','Accelerometry Y-axis', 'Accelerometry Z-axis',
                                  'Gyro X-axis','Gyro Y-axis', 'Gyro Z-axis'])
data = df1
sensor_axis_types = ['acc_x', 'acc_y', 'acc_z',
                     'gyr_x', 'gyr_y', 'gyr_z']
# Loop through each sensor axis
for i, axis in zip(range(1,7), sensor_axis_types):
    # Loop thorugh each sensor
    for sens in ['left', 'right', 'chest']:
        fig.add_trace(
        go.Scatter(x=data.index, y=data[f'{sens}_{axis}'], name=sens),
        row=i, col=1)

fig.update_layout(height=1200, width=800, title_text="Data Segment 1")
fig.show()

In [ ]:
# Mulit-line plot - separated by sensor
fig = make_subplots(rows=6, cols=1, shared_xaxes=True,
                    vertical_spacing=0.06,
                   subplot_titles=['Accelerometry Left', 'Gyro Left',
                                   'Accelerometry Right', 'Gyro Right',
                                  'Accelerometry Chest', 'Gyro Chest'])
data = df1
sensor_axis_types = ['acc_x', 'acc_y', 'acc_z',
                     'gyr_x', 'gyr_y', 'gyr_z']

count = 1
# Loop thorugh each sensor
for sens in ['left', 'right', 'chest']:
# Loop through each sensor data type
    for d_type in ['acc', 'gyr']:
        # loop through each sensor axis
        for axis in ['x', 'y', 'z']:
             fig.add_trace(
                go.Scatter(x=data.index, y=data[f'{sens}_{d_type}_{axis}'], name=sens),
                row=count, col=1)
    
        count+=1
    
    
       

fig.update_layout(height=1200, width=800, title_text="Data Segment 1 Data")
fig.show()

In [ ]:
fig = px.line(test)
fig.show()

# Dashboard

In [ ]:
# Build App
df = px.data.tips()
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("JupyterDash Demo"),
    dcc.Graph(id='graph'),
    html.Label([
        "colorscale",
        dcc.Dropdown(
            id='colorscale-dropdown', clearable=False,
            value='plasma', options=[
                {'label': c, 'value': c}
                for c in px.colors.named_colorscales()
            ])
    ]),
])# Define callback to update graph
@app.callback(
    Output('graph', 'figure'),
    [Input("colorscale-dropdown", "value")]
)
def update_figure(colorscale):
    return px.scatter(
        df, x="total_bill", y="tip", color="size",
        color_continuous_scale=colorscale,
        render_mode="webgl", title="Tips"
    )# Run app and display result inline in the notebook
app.run_server(mode='external')